# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import optuna

rng = np.random.RandomState(42)

# Data

In [2]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# Baseline

In [3]:
model = ExtraTreesRegressor(n_jobs=-1, random_state=rng)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.743285846034854e-14
Valid MAE: 3.1736202054794527


# Hyperparameter tuning

In [4]:
def load_data():
    data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
    data = pd.read_csv(data_path)
    data['DATE'] = data['DATE'].astype('datetime64[ns]')
    X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
    y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
    X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
    y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
    return X_train, X_valid, y_train, y_valid

def objective(trial):
    PARAMS = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 12),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 500),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 1.0),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.1),
    }
    X_train, X_valid, y_train, y_valid = load_data()
    model = ExtraTreesRegressor(n_jobs=-1, random_state=rng)
    model.set_params(**PARAMS)
    model.fit(X_train, y_train)
    score = np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error'))
    return score

In [5]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials = 50, # the random sampling is used instead of the TPE algorithm until the given number of trials finish in the same study
    n_ei_candidates = 24, # number of candidate samples used to calculate the expected improvement.
    multivariate = True, # multivariate TPE when suggesting candidates; default: False
    seed = 42
)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=100, interval_steps=20),
    direction='maximize',
    sampler=sampler,
    storage='sqlite:///db.sqlite3'
)
study.optimize(
    objective, n_trials=1000,
    timeout=3600*10, # in seconds
    n_jobs=1,
    show_progress_bar=True
)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-12-21 09:40:06,713] A new study created in RDB with name: no-name-88615850-b310-4eb8-860f-4fc0fe6ba7fc


  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2024-12-21 09:40:20,893] Trial 0 finished with value: -3.3421782068200194 and parameters: {'n_estimators': 406, 'max_depth': 12, 'min_samples_leaf': 366, 'max_features': 0.6387926357773329, 'min_impurity_decrease': 0.15601864044243652, 'bootstrap': True, 'ccp_alpha': 0.08661761457749352}. Best is trial 0 with value: -3.3421782068200194.
[I 2024-12-21 09:40:45,058] Trial 1 finished with value: -3.410064263128762 and parameters: {'n_estimators': 621, 'max_depth': 9, 'min_samples_leaf': 11, 'max_features': 0.9729188669457949, 'min_impurity_decrease': 0.8324426408004217, 'bootstrap': True, 'ccp_alpha': 0.01834045098534338}. Best is trial 0 with value: -3.3421782068200194.
[I 2024-12-21 09:40:53,212] Trial 2 finished with value: -3.5429537184660345 and parameters: {'n_estimators': 339, 'max_depth': 7, 'min_samples_leaf': 216, 'max_features': 0.36210622617823773, 'min_impurity_decrease': 0.6118528947223795, 'bootstrap': False, 'ccp_alpha': 0.03663618432936917}. Best is trial 0 with value:

In [6]:
print(f"Best score: {study.best_trial.value}")
print(f"Best params: {study.best_trial.params}")

Best score: -2.9926142437758414
Best params: {'n_estimators': 992, 'max_depth': 12, 'min_samples_leaf': 11, 'max_features': 0.9736956854906844, 'min_impurity_decrease': 0.0027491131982042196, 'bootstrap': False, 'ccp_alpha': 7.015288782911799e-05}


In [7]:
model = ExtraTreesRegressor(
    n_estimators=992, max_depth=12,
    min_samples_leaf=11, max_features=0.974,
    min_impurity_decrease=0.003, bootstrap=False,
    ccp_alpha=0.0,
    n_jobs=-1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.7730897245199944
Valid MAE: 3.1483803526156695


In [8]:
model = ExtraTreesRegressor(
    n_estimators=1000, max_depth=12,
    min_samples_leaf=1, max_features=1.0,
    min_impurity_decrease=0.003, bootstrap=False,
    ccp_alpha=0.0,
    n_jobs=-1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.6496171308308774
Valid MAE: 3.1497141876711154


In [9]:
model = ExtraTreesRegressor(
    n_estimators=1200, max_depth=12,
    min_samples_leaf=1, max_features=1.0,
    min_impurity_decrease=0.003, bootstrap=False,
    ccp_alpha=0.0,
    n_jobs=-1, random_state=rng
)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.6505507019293844
Valid MAE: 3.150960589947152
